<a href="https://colab.research.google.com/github/alyhussein86/C6oUVdljHlRZgnaq/blob/main/P5_ValueInvestor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from prophet import Prophet
import numpy as np
import matplotlib.pyplot as plt
from prophet.plot import plot_plotly, plot_components_plotly
from pandas import DataFrame, to_datetime
from sklearn.metrics import mean_absolute_error
from datetime import timedelta, datetime

In [ ]:
# Provide the path to your Excel file
excel_file_path = '/content/2020Q1Q2Q3Q4-2021Q1.xlsx'

# Read the Excel sheet into a pandas DataFrame
df = pd.read_excel(excel_file_path,'Russia - Sberbank Rossii PAO (S')

In [ ]:
print(df)

In [ ]:
df = df.iloc[:-1,:2]
print(df)

In [ ]:
# Filtering DF before the Q1 of second year
specific_date_str = "2021-01-01"
specific_date = pd.to_datetime(specific_date_str)

filtered_df = df[df['Date'] < specific_date]

filtered_df.rename(columns={'Date': 'ds', 'Price': 'y'}, inplace=True)

In [ ]:
print(filtered_df)

In [ ]:
# plot the time series
# filtered_df.plot()
# plt.show()
plt.plot(filtered_df['ds'])
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Current Prices')
plt.show()

In [ ]:
# Create a Prophet model
model = Prophet()

# Fit the model
model.fit(filtered_df)

In [ ]:
future = model.make_future_dataframe(periods=91)
future.tail(91)

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = model.plot(forecast)

In [ ]:
fig2 = model.plot_components(forecast)

In [ ]:
plot_plotly(model, forecast)

In [ ]:
weekend_days = [0,0,0,0,0,1,1]  # Saturday is 5, Sunday is 6

In [ ]:
start_date = specific_date.date()
end_date = (specific_date + timedelta(89)).date()

In [ ]:
print(start_date)
print(end_date)

In [ ]:
# Calculate working days by excluding weekends
business_days = np.busday_count(start_date, end_date, weekmask=weekend_days)

print("Number of business days:", business_days)

In [ ]:
forecasted = forecast[~forecast['ds'].dt.dayofweek.isin(weekend_days)]

In [ ]:
forecasted[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(64)

In [ ]:
fig3 = model.plot(forecasted)

In [ ]:
fig4 = model.plot_components(forecasted)

In [ ]:
plot_plotly(model, forecasted)

In [ ]:
y_true = df['Price'].head(64).values

print(y_true)

In [ ]:
y_pred = forecast['yhat'].tail(64).values
print(y_pred)

In [ ]:
MAE = mean_absolute_error(y_true, y_pred)

In [ ]:
print('MAE: %.3f' % MAE)

In [ ]:
# plot expected vs actual
plt.plot(y_true, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
# Calculate moving averages (adjust window size as needed)
window = 1000
forecast['rolling_mean'] = forecast['yhat'].rolling(window=window).mean()

# Generate buy/sell/hold recommendations based on forecast and moving averages
forecast['signal'] = 1  # 0: HOLD, 1: BUY, -1: SELL


In [ ]:
for i in range(window, len(forecast)):
    if forecast['yhat'][i] > forecast['rolling_mean'][i]:
        forecast['signal'][i] = 1
    elif forecast['yhat'][i] < forecast['rolling_mean'][i]:
        forecast['signal'][i] = -1

# Backtesting the strategy
capital = 100000  # Initial capital
shares_held = 0
capital_history = []

for i in range(window, len(forecast)):
    if forecast['signal'][i] == 1 and capital > 0:
        shares_held = capital / forecast['yhat'][i]
        capital = 0
    elif forecast['signal'][i] == -1 and shares_held > 0:
        capital = shares_held * forecast['yhat'][i]
        shares_held = 0

    capital_history.append(capital + shares_held * forecast['yhat'][i])

# Plot the backtest results
plt.plot(forecast['ds'][window:], capital_history)
plt.xlabel('Date')
plt.ylabel('Capital')
plt.title('Backtest Results')
plt.show()
